In [1]:
import math
import numpy as np
import numpy
from PIL import Image, ImageDraw
from PIL import ImagePath
import pandas as pd
import os
from os import path
from tqdm import tqdm
import json
import cv2
import matplotlib.pyplot as plt
import urllib

In [2]:
import tensorflow as tf
from tensorflow import keras
# tf.enable_eager_execution()
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
# from hilbert import hilbertCurve
import imgaug.augmenters as iaa
import numpy as np
# import albumentations as A
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

2023-07-18 18:58:34.299642: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-18 18:58:34.376328: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-18 18:58:35.437936: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
os.listdir('data')

['label', 'image', 'bwmask']

# Step 1: Data Preprocessing


## Load the annotated cell images and masks

In [4]:
def get_file_names(root_dir):
    image = []
    mask = []
    label= []
    for i in sorted(os.listdir(root_dir)):        
              for frame in sorted(os.listdir(path.join (root_dir,i))):
                  if i == 'bwmask':
                    mask.append(path.join (root_dir,i,frame))
                  elif i == 'label':
                    label.append(path.join(root_dir,i,frame))
                  else:
                    image.append(path.join(root_dir,i,frame))

    data_df = pd.DataFrame(data =(zip(image,mask,label)),columns = ['image','bwmask','label'])
    return data_df

In [5]:
data_df = get_file_names('data/')
data_df.tail(10)

,image,bwmask,label
43,data/image/ID319_A1_Regione-0.jp2-crop_slice_9...,data/bwmask/ID319_A1_Regione-0.jp2-crop_slice_...,data/label/ID319_A1_Regione-0.jp2-crop_slice_9...
44,data/image/ID319_A1_Regione-0.jp2-crop_slice_9...,data/bwmask/ID319_A1_Regione-0.jp2-crop_slice_...,data/label/ID319_A1_Regione-0.jp2-crop_slice_9...
45,data/image/ID319_A1_Regione-1.jp2-crop_slice_2...,data/bwmask/ID319_A1_Regione-1.jp2-crop_slice_...,data/label/ID319_A1_Regione-1.jp2-crop_slice_2...
46,data/image/ID319_A1_Regione-1.jp2-crop_slice_c...,data/bwmask/ID319_A1_Regione-1.jp2-crop_slice_...,data/label/ID319_A1_Regione-1.jp2-crop_slice_c...
47,data/image/ID319_A1_Regione-1.jp2-crop_slice_c...,data/bwmask/ID319_A1_Regione-1.jp2-crop_slice_...,data/label/ID319_A1_Regione-1.jp2-crop_slice_c...
48,data/image/ID319_A1_Regione-1.jp2-crop_slice_c...,data/bwmask/ID319_A1_Regione-1.jp2-crop_slice_...,data/label/ID319_A1_Regione-1.jp2-crop_slice_c...
49,data/image/ID343_A1b-3.jp2-crop_slice_18.png,data/bwmask/ID343_A1b-3.jp2-crop_slice_18.png,data/label/ID343_A1b-3.jp2-crop_slice_18.tif
50,data/image/ID343_A1b-3.jp2-crop_slice_6.png,data/bwmask/ID343_A1b-3.jp2-crop_slice_6.png,data/label/ID343_A1b-3.jp2-crop_slice_6.tif
51,data/image/ID343_A1b-3.jp2-crop_slice_7.png,data/bwmask/ID343_A1b-3.jp2-crop_slice_7.png,data/label/ID343_A1b-3.jp2-crop_slice_7.tif
52,data/image/ID4_A1_Regione-0-crop_slice_4.png,data/bwmask/ID4_A1_Regione-0-crop_slice_4.png,data/label/ID4_A1_Regione-0-crop_slice_4.tif


### Display the images per row

In [7]:
def strip_path(path):
  """Strips the path and returns only the file name.

  Args:
    path: The path to the file.

  Returns:
    The file name.
  """

  filename = os.path.basename(path)
  return filename


def display_images(row):
      """
      Displays all image formats in a row.

      Args:
        row: The row number of the image.

      """
    image_path = row['image']
    bwmask_path = row['bwmask']
    label_path = row['label']

    fig, axs = plt.subplots(1, 3, figsize=(10, 4))
    
    img1 = Image.open(image_path)
    imarray1 = numpy.array(img1)
    axs[0].imshow(img1)
    axs[0].set_title(strip_path(image_path))
    axs[0].axis('off')
    axs[0].text(0.5, -0.15, f'Shape: {imarray1.shape}', ha='center', transform=axs[0].transAxes)

    img2 = Image.open(bwmask_path)
    imarray2 = numpy.array(img2)
    axs[1].imshow(img2)
    axs[1].set_title(strip_path(bwmask_path))
    axs[1].axis('off')
    axs[1].text(0.5, -0.15, f'Shape: {imarray2.shape}', ha='center', transform=axs[1].transAxes)

    img3 = Image.open(label_path)
    imarray3 = numpy.array(img3)
    axs[2].imshow(img3)
    axs[2].set_title(strip_path(label_path))
    axs[2].axis('off')
    axs[2].text(0.5, -0.15, f'Shape: {imarray3.shape}', ha='center', transform=axs[2].transAxes)


    plt.tight_layout()

    plt.show()

# Display images from the first row of the DataFrame
display_images(data_df.iloc[0])


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 23)

## Extract protein localization and interaction information from the masks


In [ ]:

# Step 2: Graph Network Generation
# Create a graph data structure to represent the cell network
# Add proteins as nodes to the graph
# Add edges between interacting proteins based on the extracted interactions


In [ ]:

# Step 3: Network Analysis using Louvain Algorithm
# Use NetworkX library to implement the Louvain algorithm for community detection
# Partition the cell network into communities (protein complexes)
# Analyze the communities to identify important nodes and pathways


In [ ]:

# Step 4: Network Visualization
# Use NetworkX or other libraries like Matplotlib or Plotly to visualize the cell network
# Visualize protein complexes as separate subgraphs or colors for better understanding


In [ ]:

# Step 5: Data Integration
# Integrate additional data sources, if available, to enrich the cell network analysis
# This can include protein function data, gene expression levels, or other relevant information


In [ ]:

# Step 6: Simulation
# Implement simulation scenarios to perturb the cell network and observe its behavior
# Measure the effects of changes on the network's stability and functionality


In [ ]:

# Step 7: Optimization
# Develop optimization algorithms to find the most influential proteins or pathways
# Optimize the cell network towards a desired outcome, such as improved cell function


In [ ]:

# Step 8: Reporting
# Implement a reporting system to generate analysis and simulation reports
# Enable users to share their results and findings with others


In [ ]:

# Step 9: Evaluation Framework
# Define an evaluation framework to assess the research quality of relevant journals
# Measure impact and relevance of publications in the field of graph network analysis of cell organization


In [ ]:

# Step 10: Compile Results and Conclusions
# Summarize the findings from network analysis, simulations, and optimization
# Present conclusions about the structure and organization of proteins within a cell


In [ ]:

# Step 11: Finalize the Project
# Clean up the code, optimize performance, and refactor if necessary
# Write a detailed documentation to explain the project and its results

